<a href="https://colab.research.google.com/github/chelomec/airlines_delay_prediction/blob/main/Proyecto_DataScienceII_ML_Clasificaci%C3%B3n_Demora_Vuelos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DEMORAS EN VUELOS

##INTRODUCCIÓN

##LIBRERIAS
Se deja en este espacio las librerías que se utilizan en el proyecto.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
import kagglehub
import os

##DATASET

El dataset elegido para el desarrollo del caso son los datos de demoras de los vuelos de las aerolíneas en el año 2019 en función del clima y del aeropuerto. Este dataset se encuentra en kaggle en el siguiente link.

https://www.kaggle.com/datasets/threnjen/2019-airline-delays-and-cancellations

De igual manera, se conecta el dataset con el notebook para trabajar con el mismo.

In [ ]:
# Download latest version
dataset_dir = kagglehub.dataset_download("threnjen/2019-airline-delays-and-cancellations")

print("Path to dataset files:", dataset_dir)

# Find the CSV file within the downloaded directory
for filename in os.listdir(dataset_dir ):
    if filename.endswith("train.csv"):
        csv_file_path = os.path.join(dataset_dir, filename)
        break  # Stop searching after finding the first CSV file

# Now read the CSV file using the correct file path
df = pd.read_csv(csv_file_path)
df.head()

Path to dataset files: /root/.cache/kagglehub/datasets/threnjen/2019-airline-delays-and-cancellations/versions/9


,MONTH,DAY_OF_WEEK,DEP_DEL15,DEP_TIME_BLK,DISTANCE_GROUP,SEGMENT_NUMBER,CONCURRENT_FLIGHTS,NUMBER_OF_SEATS,CARRIER_NAME,AIRPORT_FLIGHTS_MONTH,...,PREVIOUS_AIRPORT,PRCP,SNOW,SNWD,TMAX,AWND,CARRIER_HISTORICAL,DEP_AIRPORT_HIST,DAY_HISTORICAL,DEP_BLOCK_HIST
0,7,7,0,1500-1559,3,3,26,160,American Airlines Inc.,19534,...,Chicago O'Hare International,0.00,0.0,0.0,95.0,4.25,0.237709,0.273236,0.222538,0.255479
1,4,1,0,1300-1359,4,4,63,50,SkyWest Airlines Inc.,18788,...,El Paso International,0.00,0.0,0.0,71.0,11.41,0.154651,0.121849,0.237972,0.197503
2,11,4,0,0001-0559,2,1,3,76,American Eagle Airlines Inc.,1148,...,NONE,0.00,0.0,0.0,54.0,0.45,0.117559,0.187867,0.139886,0.060327
3,3,2,0,1500-1559,7,5,14,143,Southwest Airlines Co.,7612,...,San Jose International,0.00,0.0,0.0,64.0,8.05,0.204389,0.141446,0.132868,0.202037
4,7,3,0,0800-0859,1,2,85,50,American Eagle Airlines Inc.,29376,...,Cincinnati/Northern Kentucky International,0.01,0.0,0.0,94.0,10.51,0.203263,0.193761,0.203027,0.113050


###EXPLORACIÓN DEL DATASET

Con el daset disponible, se comienza analizando el mismo para comprender la composición del mismo.

In [ ]:
df.shape

(4542343, 30)

Con shape se puede observar que el dataset se compone de más de 4 millones de registros y de 30 variables.

In [ ]:
df.dtypes

,0
MONTH,int64
DAY_OF_WEEK,int64
DEP_DEL15,int64
DEP_TIME_BLK,object
DISTANCE_GROUP,int64
SEGMENT_NUMBER,int64
CONCURRENT_FLIGHTS,int64
NUMBER_OF_SEATS,int64
CARRIER_NAME,object
AIRPORT_FLIGHTS_MONTH,int64


Con dtype se puede observar cual es el tipo de datos con los que se van a trabajar, en donde la mayoría son del tipo númerico.

In [ ]:
df.dtypes.value_counts()

,count
int64,13
float64,13
object,4


Para poder hacer un análisis preciso del dataset se debe conocer si existen valores nulos y con ello hacer el correcto tratamiento de los datos.

In [ ]:
df.isnull().sum()

,0
MONTH,0
DAY_OF_WEEK,0
DEP_DEL15,0
DEP_TIME_BLK,0
DISTANCE_GROUP,0
SEGMENT_NUMBER,0
CONCURRENT_FLIGHTS,0
NUMBER_OF_SEATS,0
CARRIER_NAME,0
AIRPORT_FLIGHTS_MONTH,0


Este dataset no contiene valores nulos, por lo tanto, no requiere en un principio el trabajo de este tipo valores.

#ANALISIS DE LA INFORMACIÓN

Como se menciona anteriormente, el objetivo del proyecto es determinar de manera predictiva si un vuelo puede demorarse, por lo cual, se debe entender si existen patrones que permitan entender las relaciones de las variables para poder llevar a cabo la predicción.

In [ ]:
correlacion = []

for col in df.columns:
  if df[col].dtype != 'object':
    corr = df[[col,'DEP_DEL15']].corr(numeric_only=True)
    correlacion.append((col,corr.iloc[0,1]))

correlacion_df = pd.DataFrame(correlacion, columns=['Column', 'Correlation'])

print(correlacion_df)

                           Column  Correlation
0                           MONTH    -0.018683
1                     DAY_OF_WEEK    -0.000030
2                       DEP_DEL15     1.000000
3                  DISTANCE_GROUP     0.015998
4                  SEGMENT_NUMBER     0.117343
5              CONCURRENT_FLIGHTS     0.009380
6                 NUMBER_OF_SEATS     0.011734
7           AIRPORT_FLIGHTS_MONTH     0.026883
8           AIRLINE_FLIGHTS_MONTH     0.003401
9   AIRLINE_AIRPORT_FLIGHTS_MONTH     0.013950
10       AVG_MONTHLY_PASS_AIRPORT     0.024386
11       AVG_MONTHLY_PASS_AIRLINE     0.001158
12        FLT_ATTENDANTS_PER_PASS    -0.002286
13           GROUND_SERV_PER_PASS    -0.016646
14                      PLANE_AGE     0.006335
15                       LATITUDE     0.000140
16                      LONGITUDE     0.027445
17                           PRCP     0.080107
18                           SNOW     0.050233
19                           SNWD     0.026275
20           

Si se considera el universo de todas las variables, en donde el dep_del15 puede ser 1 o 0 (indicando si fue mayor a 15 minutos o no) se puede observar que no existe variables significativas que influyen en si un vuelo se demora más de 15 minutos o no, pero si se puede observar que existen varibles que afectan en mayor medida que otras, siendo aquellas que no tienen relevancia descartadas del análisis, ya que no influyen en el modelo.